In [1]:
import sympy as sp
import numpy as np

状态空间模型

In [2]:
x1,x2,x3,x4 = sp.symbols('x1 x2 x3 x4')
u, mp, L, mc, g = sp.symbols('u mp L mc g')

# 定义状态空间模型
x1_dot = x3
x2_dot = -x4
x3_dot = (u + mp * sp.sin(x2) * (L * x4**2 - g * sp.cos(x2))) / (mc + mp * sp.sin(x2)**2)
x4_dot = (u * sp.cos(x2) + mp * L * x4**2 * sp.cos(x2) * sp.sin(x2) - (mc + mp) * g * sp.sin(x2)) / (L * mc + L * mp * sp.sin(x2)**2)

print(f'x1_dot: ', x1_dot)
print(f'x2_dot: ', x2_dot)
print(f'x3_dot: ', x3_dot)
print(f'x4_dot: ', x4_dot)

x1_dot:  x3
x2_dot:  -x4
x3_dot:  (mp*(L*x4**2 - g*cos(x2))*sin(x2) + u)/(mc + mp*sin(x2)**2)
x4_dot:  (L*mp*x4**2*sin(x2)*cos(x2) - g*(mc + mp)*sin(x2) + u*cos(x2))/(L*mc + L*mp*sin(x2)**2)


求偏导

In [3]:
X = [x1, x2, x3, x4]
f = [x1_dot, x2_dot, x3_dot, x4_dot]
U = [u]
# 计算雅可比矩阵（偏导数矩阵）
pfx = sp.Matrix(len(f), len(X), lambda i, j: sp.diff(f[i], X[j]))
pfu = sp.Matrix(len(f), len(U), lambda i, j: sp.diff(f[i], U[j]))

for i in range(len(f)):
    for j in range(len(X)):
        print(f'pf{i+1}x{j+1}:', pfx[i, j])
    print()

for i in range(len(f)):
    for j in range(len(U)):
        print(f'pf{i+1}u{j+1}:', pfu[i, j])
    print()


pf1x1: 0
pf1x2: 0
pf1x3: 1
pf1x4: 0

pf2x1: 0
pf2x2: 0
pf2x3: 0
pf2x4: -1

pf3x1: 0
pf3x2: -2*mp*(mp*(L*x4**2 - g*cos(x2))*sin(x2) + u)*sin(x2)*cos(x2)/(mc + mp*sin(x2)**2)**2 + (g*mp*sin(x2)**2 + mp*(L*x4**2 - g*cos(x2))*cos(x2))/(mc + mp*sin(x2)**2)
pf3x3: 0
pf3x4: 2*L*mp*x4*sin(x2)/(mc + mp*sin(x2)**2)

pf4x1: 0
pf4x2: -2*L*mp*(L*mp*x4**2*sin(x2)*cos(x2) - g*(mc + mp)*sin(x2) + u*cos(x2))*sin(x2)*cos(x2)/(L*mc + L*mp*sin(x2)**2)**2 + (-L*mp*x4**2*sin(x2)**2 + L*mp*x4**2*cos(x2)**2 - g*(mc + mp)*cos(x2) - u*sin(x2))/(L*mc + L*mp*sin(x2)**2)
pf4x3: 0
pf4x4: 2*L*mp*x4*sin(x2)*cos(x2)/(L*mc + L*mp*sin(x2)**2)

pf1u1: 0

pf2u1: 0

pf3u1: 1/(mc + mp*sin(x2)**2)

pf4u1: cos(x2)/(L*mc + L*mp*sin(x2)**2)



在x = 0，u = 0处线性化

In [4]:
x1_hat,x2_hat,x3_hat,x4_hat = 0,0,0,0
u_hat = 0
mp_value = 1
mc_value = 10
g_value = 9.81
L_value = 0.5
subs_list = [
    (x1, x1_hat), (x2, x2_hat), (x3, x3_hat), (x4, x4_hat),
    (u, u_hat), (mp, mp_value), (mc, mc_value), (g, g_value), (L, L_value)
]
A_hat = pfx.subs(subs_list).evalf()
B_hat = pfu.subs(subs_list).evalf()

#转化为numpy矩阵
A_hat = np.array(A_hat.tolist()).astype(np.float64)
B_hat = np.array(B_hat.tolist()).astype(np.float64)

print(f'A_hat:\n {A_hat}')
print(f'B_hat:\n {B_hat}')


A_hat:
 [[  0.      0.      1.      0.   ]
 [  0.      0.      0.     -1.   ]
 [  0.     -0.981   0.      0.   ]
 [  0.    -21.582   0.      0.   ]]
B_hat:
 [[0. ]
 [0. ]
 [0.1]
 [0.2]]


离散时间动力学模型

In [5]:
det_T = 0.01  # 采样时间

# 离散化
A_discrete = np.eye(4) + det_T * A_hat
B_discrete = det_T * B_hat

print(f'A_discrete:\n {A_discrete}')
print(f'B_discrete:\n {B_discrete}')

A_discrete:
 [[ 1.       0.       0.01     0.     ]
 [ 0.       1.       0.      -0.01   ]
 [ 0.      -0.00981  1.       0.     ]
 [ 0.      -0.21582  0.       1.     ]]
B_discrete:
 [[0.   ]
 [0.   ]
 [0.001]
 [0.002]]
